In [ ]:
%%capture
#from google.colab import drive
#drive.mount('/content/drive')
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
# !pip install torch==2.2.2 torchtext==0.17.2 torchvision torchaudio --force-reinstall --no-cache-dir
# !python -m spacy download en_core_web_sm

from torchtext.vocab import build_vocab_from_iterator
from nltk.translate.bleu_score import corpus_bleu

import random
import re
from collections import Counter
from tqdm import tqdm
import pickle



import numpy as np
import pandas as pd
import time

import spacy
import ast


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

In [ ]:
# 하이퍼파라미터 설정
INPUT_DIM = len(ingredient_vocab)
OUTPUT_DIM = len(recipe_vocab)
EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
DROPOUT = 0.5
LR = 0.001
BATCH_SIZE = 64
criterion = nn.CrossEntropyLoss(ignore_index=recipe_vocab['<pad>'])
TRAIN_RATIO = 1.0
EPOCHS = 10
new_model_train = True
model_type = "Seq2Seq_epoch"
save_model_path = f"/content/drive/MyDrive/FIT5127-NLP/Ass2/results/{model_type}.pt"
save_history_path = f"/content/drive/MyDrive/FIT5127-NLP/Ass2/results/{model_type}_history.pt"




'2.2.2+cu121'

In [ ]:
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
# data 다운
train_path = '/content/drive/MyDrive/FIT5127-NLP/Ass2/Cooking_Dataset/train.csv'
dev_path = '/content/drive/MyDrive/FIT5127-NLP/Ass2/Cooking_Dataset/dev.csv'
test_path = '/content/drive/MyDrive/FIT5127-NLP/Ass2/Cooking_Dataset/test.csv'

train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

# 데이터 확인
print(f"Train data size: {len(train_df)}")
print(f"Dev data size: {len(dev_df)}")
print(f"Test data size: {len(test_df)}")
print("\nTrain data sample:")
print(train_df.head())
#No-Bake Nut Cookies
# ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]
# ["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]

Train data size: 162899
Dev data size: 1065
Test data size: 1081

Train data sample:
                      Title  \
0       No-Bake Nut Cookies   
1               Creamy Corn   
2      Reeses Cups(Candy)     
3  Cheeseburger Potato Soup   
4       Rhubarb Coffee Cake   

                                         Ingredients  \
0  ["1 c. firmly packed brown sugar", "1/2 c. eva...   
1  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
2  ["1 c. peanut butter", "3/4 c. graham cracker ...   
3  ["6 baking potatoes", "1 lb. of extra lean gro...   
4  ["1 1/2 c. sugar", "1/2 c. butter", "1 egg", "...   

                                              Recipe  
0  ["In a heavy 2-quart saucepan, mix brown sugar...  
1  ["In a slow cooker, combine all ingredients. C...  
2  ["Combine first four ingredients and press in ...  
3  ["Wash potatoes; prick several times with a fo...  
4  ["Cream sugar and butter.", "Add egg and beat ...  


In [ ]:
def tokenizer(text, remove_stopwords=True, lemmatize=True):
    text_list = ast.literal_eval(text)
    tokens = []

    with spacy_en.select_pipes(disable=["tagger", "parser", "ner"]):

        for item in text_list:
            doc = spacy_en(item.lower())

            for token in doc:
                # 숫자 제거
                if token.is_digit:
                    continue

                # 단위 제거 (c., tbsp., tsp., oz., lb.)
                if any(unit in token.text for unit in ['c.', 'tbsp.', 'tsp.', 'oz.', 'lb.']):
                    continue

                # 불용어 제거 (선택)
                if remove_stopwords and token.is_stop:
                    continue

                # 레마타이징 또는 원형 유지
                if lemmatize:
                    tokens.append(token.lemma_)
                else:
                    tokens.append(token.text)
    return tokens


["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]
['1', 'c.', 'firmly', 'packed', 'brown', 'sugar', '1/2', 'c.', 'evaporated', 'milk', '1/2', 'tsp', '.', 'vanilla', '1/2', 'c.', 'broken', 'nuts', '(', 'pecans', ')', '2', 'tbsp', '.', 'butter', 'margarine', '3', '1/2', 'c.', 'bite', 'size', 'shredded', 'rice', 'biscuits']


/usr/local/lib/python3.11/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
def build_vocab(token_lists, min_freq=2):
    def yield_tokens():
        for tokens in token_lists:
            yield tokens

    vocab = build_vocab_from_iterator(yield_tokens(), min_freq=min_freq,
                                      specials=['<pad>', '<sos>', '<eos>', '<unk>'])
    vocab.set_default_index(vocab['<unk>'])  # unknown token 처리
    return vocab


In [ ]:
# 캐시된 파일명
ingredient_cache_path = "/content/drive/MyDrive/FIT5127-NLP/Ass2/ingredient_tokens.pkl"
recipe_cache_path = "/content/drive/MyDrive/FIT5127-NLP/Ass2/recipe_tokens.pkl"

# 캐시가 있다면 불러오기, 없으면 토큰화해서 저장
try:
    with open(ingredient_cache_path, "rb") as f:
        ingredient_token_lists = pickle.load(f)
    with open(recipe_cache_path, "rb") as f:
        recipe_token_lists = pickle.load(f)
    print("Successfully loaded the cached token list!")
except FileNotFoundError:
    print("No cache found → Starting tokenization...")
    ingredient_token_lists = [tokenizer(text) for text in tqdm(train_df['Ingredient'], desc="Tokenizing ingredients")]
    recipe_token_lists = [tokenizer(text) for text in tqdm(train_df['Recipe'], desc="Tokenizing recipes")]
    # 저장
    with open(ingredient_cache_path, "wb") as f:
        pickle.dump(ingredient_token_lists, f)
    with open(recipe_cache_path, "wb") as f:
        pickle.dump(recipe_token_lists, f)
    print("Saved token list!")



# Vocab 생성 (2번 이상 등장한 단어만 포함)
ingredient_vocab = build_vocab(ingredient_token_lists)
recipe_vocab = build_vocab(recipe_token_lists)

# 확인
print("Ingredient vocab size:", len(ingredient_vocab))
print("Recipe vocab size:", len(recipe_vocab))

# 예시로 일부 단어 출력
print("Sample ingredient tokens:", list(enumerate(ingredient_vocab.get_itos()))[:10])
print("Sample recipe tokens:", list(enumerate(recipe_vocab.get_itos()))[:10])



✅ 캐시된 토큰 리스트 불러오기 완료!
Ingredient vocab size: 5756
Recipe vocab size: 8364
Sample ingredient tokens: [(0, '<pad>'), (1, '<sos>'), (2, '<eos>'), (3, '<unk>'), (4, '1'), (5, '.'), (6, 'c.'), (7, '2'), (8, '1/2'), (9, ',')]
Sample recipe tokens: [(0, '<pad>'), (1, '<sos>'), (2, '<eos>'), (3, '<unk>'), (4, '.'), (5, ','), (6, 'add'), (7, 'mix'), (8, 'minutes'), (9, ';')]


In [ ]:
class CustomDataset(Dataset):
    def __init__(self,df,ingredient_vocab,recipe_vocab):
        self.df = df
        self.ingredient_vocab = ingredient_vocab
        self.recipe_vocab = recipe_vocab

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 전처리된 텍스트 토큰화
        ingredient_tokens = tokenizer(self.df.iloc[idx]["Ingredients"])
        recipe_tokens = tokenizer(self.df.iloc[idx]["Recipe"])

        # 인덱스로 변환
        ingredient_ids = [self.ingredient_vocab[token] for token in ingredient_tokens]
        recipe_ids = [self.recipe_vocab['<sos>']] + [self.recipe_vocab[token] for token in recipe_tokens ]+[self.recipe_vocab['<eos>']]

        # 텐서로
        return torch.tensor(ingredient_ids), torch.tensor(recipe_ids)

In [ ]:
def collate_fn(batch):
    ingredients, recipes = zip(*batch)
    ingredients_padded = pad_sequence(ingredients, batch_first=True, padding_value=ingredient_vocab['<pad>'])
    recipes_padded = pad_sequence(recipes, batch_first=True, padding_value=recipe_vocab['<pad>'])
    return ingredients_padded.to(DEVICE), recipes_padded.to(DEVICE)


In [ ]:
train_dataset = CustomDataset(train_df, ingredient_vocab, recipe_vocab)
dev_dataset = CustomDataset(dev_df, ingredient_vocab, recipe_vocab)
test_dataset = CustomDataset(test_df, ingredient_vocab, recipe_vocab)


train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# 확인
sample_ingredients, sample_recipes = next(iter(train_loader))
print("Ingredients batch shape:", sample_ingredients.shape)
print("Recipes batch shape:", sample_recipes.shape)


Ingredients batch shape: torch.Size([4, 74])
Recipes batch shape: torch.Size([4, 75])


In [ ]:
class Encoder_GRU(nn.Module):
    def __init__(self, ingredient_vocab_size, embedding_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()
        # 임베딩
        self.embedding = nn.Embedding(ingredient_vocab_size,embedding_dim)

        # GRU 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(embedding_dim,
                          hidden_dim,
                          n_layers,
                          dropout=dropout_ratio if n_layers>1 else 0,
                          batch_first=True)

        # 드롭아웃
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, src):
        # src : [batch_size, src_len]

        # 임베딩
        embedded = self.dropout(self.embedding(src))
        # embedded : [batch_size, src_len, hidden_dim]

        # gru 통과
        outputs, hidden = self.gru(embedded) # h0를 따로 주지 않으면, 디폴트로 h0가 0로 초기화되서 들어감
        # outputs: [batch_size, src_len, hidden_size]
        # hidden: [n_layers, batch_size, hidden_size]

        return hidden

class Decoder_GRU(nn.Module):
    def __init__(self, recipe_vocab_size, embedding_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        self.recipe_vocab_size = recipe_vocab_size

        # 임베딩
        self.embedding = nn.Embedding(recipe_vocab_size, embedding_dim)

        # GRU 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(embedding_dim,
                          hidden_dim,
                          n_layers,
                          dropout=dropout_ratio if n_layers>1 else 0,
                          batch_first=True)

        # fc 레이어
        self.fc_out = nn.Linear(hidden_dim, recipe_vocab_size)

        # 드롭아웃
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, input, hidden):
        # input : [batch_size]
        input = input.unsqueeze(1)
        # input : [batch_size, 단어의 개수=1]

        # 임베딩
        embedded = self.dropout(self.embedding(input))
        # embedded : [batch_size, 단어의 개수=1, hidden_dim]

        # GRU 통과
        outputs, hidden = self.gru(embedded,hidden)
        # outputs: [batch_size, 단어의 개수=1, hidden_size]
        # hidden: [n_layers, batch_size, hidden_size]

        # fc 통과
        prediction = self.fc_out(outputs.squeeze(1))
        # prediction: [batch_size, vocab_size]

        return prediction, hidden


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, target=None, teacher_forcing_ratio=0.5, max_len=50):
        hidden = self.encoder(src)
        batch_size = src.size(0)
        vocab_size = self.decoder.recipe_vocab_size

        # 🟢 추론 모드
        if target is None:
            outputs = []
            input_token = torch.tensor([recipe_vocab['<sos>']] * batch_size).to(self.device)

            for _ in range(max_len):
                output, hidden = self.decoder(input_token, hidden)
                top1 = output.argmax(1)
                outputs.append(top1.unsqueeze(1))
                input_token = top1

            return torch.cat(outputs, dim=1)  # [batch_size, max_len]

        # 🟢 학습 모드
        target_len = target.shape[1]
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)
        input_token = target[:, 0]  # <sos>

        for t in range(1, target_len):
            output, hidden = self.decoder(input_token, hidden)
            outputs[:, t, :] = output
            top1 = output.argmax(1)
            teacher_force = random.random() < teacher_forcing_ratio
            input_token = target[:, t] if teacher_force else top1

        return outputs




In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def compute_bleu(model, dataloader, recipe_vocab, max_len=50):
    """
    Seq2Seq 모델의 BLEU 점수를 계산 (1~4-gram 기준)

    Args:
        model: 학습된 Seq2Seq 모델
        dataloader: 검증 또는 테스트 DataLoader
        recipe_vocab: recipe vocab 객체 (인덱스 → 단어 변환용)
        max_len: 생성할 최대 문장 길이

    Returns:
        bleu_score (float): 0~100 사이의 BLEU 점수
    """
    model.eval()
    preds = []
    targets = []

    with torch.no_grad():
        for src_batch, trg_batch in tqdm(dataloader, desc="Evaluating BLEU"):
            src_batch = src_batch.to(DEVICE)
            trg_batch = trg_batch.to(DEVICE)

            # 생성된 예측 시퀀스: [batch_size, max_len]
            generated = model(src_batch, target=None, teacher_forcing_ratio=0.0, max_len=max_len)

            for i in range(src_batch.size(0)):
                pred_tokens = generated[i].tolist()
                trg_tokens = trg_batch[i].tolist()

                # <eos> 토큰이 나오면 거기서 자름
                if recipe_vocab['<eos>'] in pred_tokens:
                    pred_tokens = pred_tokens[:pred_tokens.index(recipe_vocab['<eos>'])]
                if recipe_vocab['<eos>'] in trg_tokens:
                    trg_tokens = trg_tokens[:trg_tokens.index(recipe_vocab['<eos>'])]

                preds.append(pred_tokens)
                targets.append([trg_tokens])  # corpus_bleu는 list of list 필요

    bleu = corpus_bleu(targets, preds) * 100
    print(f"\nBLEU Score: {bleu:.2f}")
    return bleu


def loss_epoch_seq2seq(model, dataloader, criterion, optimizer=None, teacher_forcing_ratio=0.5):
    """
    학습 또는 평가 루프에서 1 epoch 동안 loss만 계산하는 함수.

    Args:
        model: Seq2Seq 모델
        dataloader: DataLoader
        criterion: 손실 함수 (CrossEntropyLoss)
        optimizer: 옵티마이저 (None이면 eval 모드로 동작)
        teacher_forcing_ratio: 학습 중 Teacher Forcing 비율

    Returns:
        epoch_loss: 전체 데이터셋에 대한 평균 loss
    """
    model.train() if optimizer else model.eval()
    rloss = 0

    for src_batch, trg_batch in tqdm(dataloader, leave=False):
        src_batch = src_batch.to(DEVICE)
        trg_batch = trg_batch.to(DEVICE)

        output = model(src_batch, trg_batch, teacher_forcing_ratio=teacher_forcing_ratio)
        output_dim = output.shape[-1]

        # CrossEntropyLoss 계산을 위한 reshape
        output = output[:, 1:, :].reshape(-1, output_dim)
        trg = trg_batch[:, 1:].reshape(-1)

        loss = criterion(output, trg)

        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()

        rloss += loss.item() * src_batch.shape[0]

    return rloss / len(dataloader.dataset)

def Train(model, train_loader, val_loader, criterion, optimizer,
          EPOCHS, BATCH_SIZE, TRAIN_RATIO,
          save_model_path, save_history_path,
          ):
    """
    이어서 학습할 수 있게 start_epoch와 best_val_loss를 인자로 받음
    """

    loss_history = {"train": [], "val": []}
    best_val_loss = float('inf')
    train_start_time = time.time()
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")

        ep_start_time = time.time()

        # Training
        train_loss = loss_epoch_seq2seq(model, train_loader, criterion, optimizer, teacher_forcing_ratio=0.5)
        loss_history["train"].append(train_loss)

        # Validation
        val_loss = loss_epoch_seq2seq(model, val_loader, criterion, optimizer=None, teacher_forcing_ratio=0.0)
        loss_history["val"].append(val_loss)

        ep_elapsed_time = time.time() - ep_start_time

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": epoch,
                "val_loss": val_loss,
                "train_loss": train_loss,  # train_loss 저장
            }, save_model_path)
            print("Best model saved!")

        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Time: {ep_elapsed_time:.2f}s")
    train_elapsed_time = time.time() - train_start_time
    # Save training history
    torch.save({
        "loss_history": loss_history,
        "EPOCHS": EPOCHS,
        "BATCH_SIZE": BATCH_SIZE,
        "TRAIN_RATIO": TRAIN_RATIO
    }, save_history_path)

    print(f"\nTraining Completed! History saved!| Elapsed Time : {train_elapsed_time}")
    return loss_history


def Test(model, test_loader, criterion, recipe_vocab):
    """
    테스트 데이터셋에서 손실과 BLEU 점수를 계산하는 함수.

    Args:
        model: 학습된 Seq2Seq 모델
        test_loader: 테스트 DataLoader
        criterion: 손실 함수
        recipe_vocab: vocab 객체 (BLEU 계산용)

    Returns:
        test_loss (float), bleu_score (float)
    """
    print("\n Testing on test set...")
    model.eval()

    with torch.no_grad():
        test_loss = loss_epoch_seq2seq(model, test_loader, criterion, optimizer=None, teacher_forcing_ratio=0.0)

    bleu_score = compute_bleu(model, test_loader, recipe_vocab)
    print(f"Test Loss: {test_loss:.4f} | BLEU Score: {bleu_score:.2f}")

    return test_loss, bleu_score




In [ ]:
# 모델 생성
encoder = Encoder_GRU(INPUT_DIM, EMBED_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT)
decoder = Decoder_GRU(OUTPUT_DIM, EMBED_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT)
model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

if new_model_train:
    print("New model training!")
    optimizer = optim.Adam(model.parameters(), lr=LR)

    loss_history = Train(
        model=model,
        train_loader=train_loader,
        val_loader=dev_loader,
        criterion=criterion,
        optimizer=optimizer,
        EPOCHS=EPOCHS,
        BATCH_SIZE=BATCH_SIZE,
        TRAIN_RATIO=TRAIN_RATIO,
        save_model_path=save_model_path,
        save_history_path=save_history_path,
    )

    # 플롯 그리기
    plt.figure(figsize=(10,6))
    plt.plot(range(1, EPOCHS+1), loss_history["train"], label="Train Loss", color="blue")
    plt.plot(range(1, EPOCHS+1), loss_history["val"], label="Validation Loss", color="red", linestyle='--')
    plt.xlabel("Epoch", fontsize=18)
    plt.ylabel("Loss", fontsize=18)
    plt.title("Training and Validation Loss", fontsize=20)
    plt.legend(fontsize=14)
    plt.grid()
    plt.show()


🚀 이어서 학습 시작합니다!
✅ Previous Epoch 1 Result:
Val Loss: 5.1952

Epoch 2/10


  8%|▊         | 3372/40725 [12:40<1:59:55,  5.19it/s]

In [ ]:
saved_model = torch.load(save_model_path, map_location=DEVICE)
# 모델 다시 생성 (구조만 똑같이)
encoder = Encoder_GRU(INPUT_DIM, EMBED_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT)
decoder = Decoder_GRU(OUTPUT_DIM, EMBED_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT)
load_model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)
# state_dict 로드
load_model.load_state_dict(saved_model["model_state_dict"])

In [ ]:
# ✅ 이제 Test 가능
Test(load_model, test_loader, criterion, recipe_vocab)